In [3]:
import pandas as pd

df = pd.read_csv("final_merged_dataset.csv")

print(df.head())


         Date  Gold_Price  Open_x  High_x  Low_x  Volume  Chg%  USD_Price  \
0  2020-07-17       49001   48812   49086  48760   28010  0.39      95.94   
1  2020-07-16       48811   49121   49246  48755   30040 -0.75      96.35   
2  2020-07-15       49182   49240   49273  48951   37680 -0.19      96.08   
3  2020-07-14       49277   49150   49299  48811   41500  0.20      96.26   
4  2020-07-13       49180   48920   49258  48920   34250  0.61      96.46   

   Open_y  High_y  Low_y Change %  avg_sentiment  news_count  
0   96.28   96.33  95.90   -0.42%      -0.042557          68  
1   96.00   96.40  95.89    0.28%      -0.060971         113  
2   96.17   96.25  95.78   -0.18%       0.009726         119  
3   96.54   96.70  96.18   -0.21%      -0.029302         111  
4   96.66   96.68  96.27   -0.19%       0.047699          80  


In [6]:
import pandas as pd
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")
df.set_index("Date", inplace=True)



In [8]:
features = ["Gold_Price", "USD_Price", "avg_sentiment", "news_count"]
df = df[features]


In [10]:
!pip install scikit-learn


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/8.1 MB 2.8 MB/s eta 0:00:03
   ------- -------------------------------- 1.6/8.1 MB 3.0 MB/s eta 0:00:03
   ------------ --------------------------- 2.6/8.1 MB 3.5 MB/s eta 0:00:02
   ------------------ --------------------- 3.7/8.1 MB 3.9 MB/s eta 0:00:02
   ------------------------ --------------- 5.0/8.1 MB 4.3 MB/s eta 0:00:01
   -------------------------------- ------- 6.6/8.1 MB 4.8 MB/s eta 0:00:01
   -------------------------------------- - 7.9/8.1 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 5.1 MB/s  0:00:01
   ---------------------------------------- 0.0/37.1 MB ? eta -:--:--
   - -------------------------------------- 1.6/37.1 MB 7.7 MB/s eta 0:00:05
   --- ------------------------------------ 3.1/37.1 MB 7.9 MB/s eta 0:00:05
   ----- ------------------------

In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

print(scaled_data.shape)


(654, 4)


In [12]:
import numpy as np

X = []
y = []

window_size = 30

for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i-window_size:i])
    y.append(scaled_data[i, 0])  # Gold price column

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (624, 30, 4)
y shape: (624,)


In [13]:
test_size = 30

X_train = X[:-test_size]
X_test = X[-test_size:]

y_train = y[:-test_size]
y_test = y[-test_size:]

print("Train:", X_train.shape)
print("Test:", X_test.shape)


Train: (594, 30, 4)
Test: (30, 30, 4)


In [21]:
# Train-Test Split (Time-based)
train_size = int(len(df) * 0.8)

train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Scaling (Correct Way)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

feature_cols = ['Gold_Price','USD_Price','avg_sentiment','news_count']


train_scaled = scaler.fit_transform(train_data[feature_cols])
test_scaled = scaler.transform(test_data[feature_cols])

print(train_scaled.shape)
print(test_scaled.shape)


(523, 4)
(131, 4)


In [22]:
import numpy as np

def create_sequences(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(data[i, 0])  # Gold_Price
    return np.array(X), np.array(y)

window_size = 60

X_train, y_train = create_sequences(train_scaled, window_size)
X_test, y_test = create_sequences(test_scaled, window_size)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


X_train shape: (463, 60, 4)
y_train shape: (463,)
